# Dealing with Multiple LHE Files

Oftentimes, you may wish to juggle many LHE files that have been generated using the same (or extremely similar) methods and you wish to combine all of these LHE files into one "sample" which you can analyze with a single set of analysis code. This can be done rather easily and quickly by utilizing an intermediate parquet file which is supported by [awkward](https://awkward-array.org/doc/main/user-guide/how-to-convert-arrow.html).

In [2]:
import pylhe
import awkward as ak

# Use an example LHE file from package scikit-hep-testdata
from skhep_testdata import data_path
lhe_file = data_path("pylhe-drell-yan-ll-lhe.gz")

# our input files will simply be multiple copies of the same file for this example,
# but you can imagine doing the same process below with actually different LHE files
list_of_input_files = [ lhe_file for _ in range(3) ]

# get arrays for each file
unmerged_arrays = [pylhe.to_awkward(pylhe.read_lhe_with_attributes(f)) for f in list_of_input_files]
# merge arrays into single mega-array
array = ak.concatenate(unmerged_arrays)
# store merged array into cache parquet file
ak.to_parquet(array, 'merged.parquet')
# any below analysis code can retrieve array using ak.from_parquent('merged.parquet')

  created_by: parquet-cpp-arrow version 12.0.0
  num_columns: 19
  num_rows: 30000
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

Now all the analysis code can utilize the merged file which only needs to be regenerated if more files want to be included or the source LHE files change.

In [4]:
ak.from_parquet('merged.parquet')

<EventArray [{eventinfo: {...}, ...}, ..., {...}] type='30000 * Event[event...'>